# ConvNeXt
upload transposed2.Rds

In [1]:
import rpy2.robjects as ro
from rpy2.robjects import r
import torch
from rpy2.robjects import pandas2ri
from torch.nn import Conv2d
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torch import nn
from rpy2.robjects.conversion import localconverter

torch.manual_seed(88888888)

DATA_PATH = 'transposed2.Rds' #'/home/jgburk/PycharmProjects/IntroductionToDeepLearning/transposed.Rds'
NUM_FEATURES = 1000
NUM_CLASSES = 7
NUM_TRAIN_IT = 1000

CNVNXT_CHNLS = 3
CNVNXT_CST_D1 = 18
CNVNXT_CST_D2 = 19

assert (CNVNXT_CHNLS * CNVNXT_CST_D1 * CNVNXT_CST_D2) > NUM_FEATURES

BATCH_SIZE = 32


class PandasDataset(Dataset):

    def __init__(self, rds_fn):
      with localconverter(ro.default_converter + pandas2ri.converter):
        df = ro.conversion.rpy2py(r.readRDS(rds_fn))

        features = df.iloc[:, 0:NUM_FEATURES].values
        targets = df['sample_type'].values

        unpadded = torch.tensor(features)
        padded = torch.nn.ConstantPad1d((0, (CNVNXT_CHNLS * CNVNXT_CST_D1 * CNVNXT_CST_D2) - NUM_FEATURES), 0)(unpadded)
        self.x = padded
        self.y = torch.tensor(targets - 1)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, i):
        return self.x[i], self.y[i]


def train(loader, dv):
    model.train()

    correct = 0
    for batch in loader:  # Iterate in batches over the training dataset.
        x = batch[0].reshape(BATCH_SIZE, CNVNXT_CHNLS, CNVNXT_CST_D1, CNVNXT_CST_D2).to(dv)
        y = batch[1].to(dv)
        out = model(x.float())  # Perform a single forward pass.
        y = torch.squeeze(y)
        loss = criterion(out, y.long())  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


rds_dataset = PandasDataset(DATA_PATH)
training_loader = DataLoader(rds_dataset, batch_size=BATCH_SIZE, shuffle=False)

model = models.convnext_tiny()
conv2d = model._modules['features'][6][1]
model._modules['features'][6][1] = Conv2d(in_channels=conv2d.in_channels,
                                          out_channels=conv2d.out_channels,
                                          kernel_size=conv2d.kernel_size,
                                          stride=conv2d.stride,
                                          padding=1)
device = torch.device('cuda')
model.to(device)

optimizer = torch.optim.AdamW(model.parameters())
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(NUM_TRAIN_IT):
    train(training_loader, device)
    train_acc = train(training_loader, device)
    print(f'Epoch: {epoch}, Train Acc: {train_acc}')
    if train_acc == 1.0:
        break

Epoch: 0, Train Acc: 0.034722222222222224
Epoch: 1, Train Acc: 0.013888888888888888
Epoch: 2, Train Acc: 0.14583333333333334
Epoch: 3, Train Acc: 0.04861111111111111
Epoch: 4, Train Acc: 0.13194444444444445
Epoch: 5, Train Acc: 0.2847222222222222
Epoch: 6, Train Acc: 0.3402777777777778
Epoch: 7, Train Acc: 0.28125
Epoch: 8, Train Acc: 0.2916666666666667
Epoch: 9, Train Acc: 0.3263888888888889
Epoch: 10, Train Acc: 0.3645833333333333
Epoch: 11, Train Acc: 0.1423611111111111
Epoch: 12, Train Acc: 0.3819444444444444
Epoch: 13, Train Acc: 0.3402777777777778
Epoch: 14, Train Acc: 0.4270833333333333
Epoch: 15, Train Acc: 0.5347222222222222
Epoch: 16, Train Acc: 0.4722222222222222
Epoch: 17, Train Acc: 0.5555555555555556
Epoch: 18, Train Acc: 0.5034722222222222
Epoch: 19, Train Acc: 0.6284722222222222
Epoch: 20, Train Acc: 0.5277777777777778
Epoch: 21, Train Acc: 0.5902777777777778
Epoch: 22, Train Acc: 0.4930555555555556
Epoch: 23, Train Acc: 0.3784722222222222
Epoch: 24, Train Acc: 0.496527